### Game mit Eventloop
Wir benutzen einen Eventloop, um ein einfaches snake-artiges Spiel zu programmieren,
bei dem der Spieler einen Linienzug zeichnet. Die Linie setzt sich jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.
Kreuzt sich die Linie oder erreicht sie den Rand der Leinwand, endet das Spiel.

Wir denken uns die Linie als Schlange, die sich in einem NxN Gitter bewegt und immer länger wird.
Der Spielzustand ist in einem Dict gespeichert.
Die Position des Kopfes `(col, row)` ist dem Schlüssel `head` zugewiesen,
die vom restlichen Körper besetzten Felder sind als
NxN Tabelle repräsentiert, die dem Schlüssel `body` zugewiesen ist.
Besetzte Felder sind mit `True` markiert, leere mit `False`.
Die Spielkomponente hat eine Funktion `move(dx, dy)`, welche die aktuelle Kopfposition `(x, y)`,
auf `(x+dx, y+dy)` updated und `(x, y)` als besetzt markiert.

Der Spielzustand wird auf der Leinwand dargestellt. Dabei wird in der Mitte des Feldes eine Linie in Bewegungsrichtung gezeichnet.

Wir implementieren zuerst eine Variante des Spiels, bei dem sich die Schlange nur bewegt, wenn eine Pfeiltaste gedrückt wird.

***
Die Spielkomponente unseres Spiels ist in `game.py` implementiert.  
Nachstehende testen wir diese, indem wir ein neues Spiel starten und dann
versuchen, ein Quadrat zweimal zu umrunden.  
`game.output` sagt uns, was vorgeht. Nach einer Runde kreuzt sich die Linie und
es ist Game-Over. 
***

In [ ]:
import game

# Right, Up, Left, Down -> Game over
moves = [(1, 0), (0, 1), (-1, 0), (0, -1)] * 2

game.new_game(nrow=5, ncol=5)
for dx, dy in moves:
    game.move(dx, dy)

game.state

### Darstellunskomponente
Als nächstes kümmern wir uns um die Darstellung.
Das ist relativ simple (`siehe dastellung_game.py`).
Die aktuelle Position des Kopfs ist im Dict `state` gespeichert.
Für jedes update-Event der Spielkomponente schreiben wir eine
gleichnamige Funktion, welche die Änderung auf der Leinwand vornimmt.  
`head = (col, row)` ist die Kopfposition in Gitterkoordinaten,
`size=(ncol, rnow)` sind die Abmessungen des Gitters.
Die Hilfsfunktion `grid2canvas(head, size)` rechnet Gitterkoordinaten im Canvaskoordinaten um.
- `new_game(head, size)`: Der Kopf wird als Punkt an der entsprechenden Stelle auf der
Leinwand gezeichnet.
- `game_over()`: schreibt Game-Over auf die Leinwand
- `move(head, size)`: Zeichnet eine Linie von der vorherigen zur aktuellen Kopfposition,
  und setzt `state['head'] = head`.

  

**Bemerkung**:
Bereits importierte Module werden bei erneutem Ausführen der Import-Anweisung nicht erneut importiert. 
Damit wird nur beim ersten Import von `darstellng_game` eine neue Leinwand erstellt.
Damit
`canvas.on_key_down(on_key_down)` nicht bei jedem Ausführen eine weitere Instanz der Funktion `on_key_down` als Callback registriert, ist
der Kernel jeweils neu zu starten, um erneutes Importieren zu erzwingen.


Nachstehen testen wir die Funktionen der Darstellungskomponente.

In [ ]:
import darstellung_game as D



D.canvas

In [ ]:
size = (19, 19)
D.new_game(head=(9, 9), size=size)

In [ ]:
heads = [(10, 9), (10, 10), (9, 10), (9, 9)] 
for head in heads:
    D.move(head, size)

In [ ]:
D.game_over()

### Spiel- und Darstellungskomponente
**Starte** vor Ausführen der nächsten  Zelle den **Kernel neu**, um erneutes Importieren zu erzwingen, damit eine neue Canvas verwendet wird.  

Mit `n` wird das Spiel gestartet und mit den Pfeiltasten die Fortsetzungsrichtung der Linie gesteuert.

In [ ]:
import game
import darstellung_game as D
from ipywidgets import Output
from IPython.display import display

canvas = D.canvas  # um nicht immer D.canvas schreiben zu muessen
layout = {'border': '1px solid black'}
out = Output(layout=layout)

key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
    if key == 'n':
        game.new_game()


def update(event, **kwargs):
    # print(event, kwargs)
    if event == 'new_game':
        D.new_game(**kwargs)
    if event == 'game_over':
        D.game_over()
    if event == 'move':
        D.move(**kwargs)


game.update = update
canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

***
Fortsetzung im Notebook `Game_mit_Eventloop_2.ipynb`
***